In [4]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

# Set up paths
# base_dir = "C:/Photos"
# base_dir = "C:/OneDrive/! Juxer/TVDI/趙雲瀚老師/Workplace/Fan_AOI_Austin/Photos"
# C:\OneDrive\! Juxer\TVDI\趙雲瀚老師\Workplace\Project\Fan_AOI_Austin
base_dir = "Photos"

# Data augmentation and normalization for training
train_datagen = ImageDataGenerator(rescale=1.0/255.0, validation_split=0.2)  # 20% for validation

# Load training data
train_generator = train_datagen.flow_from_directory(
    base_dir,
    target_size=(80, 80),
    batch_size=32,
    class_mode='binary',  # 'binary' because it's a 2-class problem (GO/NG)
    subset='training'
)

# Load validation data
validation_generator = train_datagen.flow_from_directory(
    base_dir,
    target_size=(80, 80),
    batch_size=32,
    class_mode='binary',
    subset='validation'
)

# Define a simple CNN model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(80, 80, 3)),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(1, activation='sigmoid')  # Use 'sigmoid' for binary classification
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(
    train_generator,
    epochs=10,
    validation_data=validation_generator
)

# Save the trained model
model.save("fan_model.h5")


Found 270 images belonging to 2 classes.
Found 66 images belonging to 2 classes.


c:\Users\teyin\miniconda3\envs\ForML\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
c:\Users\teyin\miniconda3\envs\ForML\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
9/9 ━━━━━━━━━━━━━━━━━━━━ 5s 386ms/step - accuracy: 0.5913 - loss: 1.0613 - val_accuracy: 0.6212 - val_loss: 0.6702
Epoch 2/10
9/9 ━━━━━━━━━━━━━━━━━━━━ 3s 325ms/step - accuracy: 0.6902 - loss: 0.5869 - val_accuracy: 0.6212 - val_loss: 0.6471
Epoch 3/10
9/9 ━━━━━━━━━━━━━━━━━━━━ 3s 278ms/step - accuracy: 0.7423 - loss: 0.5043 - val_accuracy: 0.4091 - val_loss: 0.6280
Epoch 4/10
9/9 ━━━━━━━━━━━━━━━━━━━━ 2s 254ms/step - accuracy: 0.8633 - loss: 0.3652 - val_accuracy: 0.8939 - val_loss: 0.5123
Epoch 5/10
9/9 ━━━━━━━━━━━━━━━━━━━━ 2s 253ms/step - accuracy: 0.9434 - loss: 0.2007 - val_accuracy: 0.3788 - val_loss: 0.6736
Epoch 6/10
9/9 ━━━━━━━━━━━━━━━━━━━━ 2s 265ms/step - accuracy: 0.9881 - loss: 0.1184 - val_accuracy: 0.3788 - val_loss: 0.7778
Epoch 7/10
9/9 ━━━━━━━━━━━━━━━━━━━━ 2s 256ms/step - accuracy: 0.9676 - loss: 0.0971 - val_accuracy: 0.6364 - val_loss: 0.6366
Epoch 8/10
9/9 ━━━━━━━━━━━━━━━━━━━━ 2s 251ms/step - accuracy: 0.9697 - loss: 0.0775 - val_accuracy: 0.8939 - val_loss:

In [5]:
import cv2
import numpy as np
import tensorflow as tf
import glob
import os

# Load the pre-trained model
model = tf.keras.models.load_model("fan_model.h5")

# Recompile the model to suppress the warning
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Function to classify a single image
def classify_image(image_path):
    img = cv2.imread(image_path)
    if img is None:
        print(f"Error: Unable to read image at {image_path}")
        return None, None
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, (80, 80))
    img = img.reshape(1, 80, 80, 3) / 255.0  # Normalize
    prediction = model.predict(img)
    
    # Handle cases where the prediction might not be well-formed
    if prediction is not None and len(prediction) > 0:
        class_idx = 1 if prediction[0] >= 0.5 else 0  # Threshold for binary classification
        confidence = prediction[0][0] * 100  # Confidence score in percentage
        return class_idx, confidence
    else:
        return None, None

# Function to classify all images in given directories
def classify_images_in_directories(directories):
    for directory_path in directories:
        print(f"\nClassifying images in directory: {directory_path}")
        
        # Use glob to find all image files (supports .jpg, .png, .jpeg)
        image_files = glob.glob(os.path.join(directory_path, "*.jpg")) + \
                      glob.glob(os.path.join(directory_path, "*.png")) + \
                      glob.glob(os.path.join(directory_path, "*.jpeg"))
        
        if not image_files:
            print("No images found in the directory!")
            continue
        
        for image_path in image_files:
            result, confidence = classify_image(image_path)
            if result is not None:
                if result == 1:
                    print(f"GO (Correctly Aligned) with {confidence:.2f}% confidence for image: {os.path.basename(image_path)}")
                else:
                    print(f"NG (Defective) with {confidence:.2f}% confidence for image: {os.path.basename(image_path)}")
            else:
                print(f"Unable to classify image: {os.path.basename(image_path)}")

# Example usage: Classify images from both "60" and "100" folders
# directories = ["C:/OneDrive/! Juxer/TVDI/趙雲瀚老師/Workplace/Fan_AOI_Austin/Photos/60", "C:/OneDrive/! Juxer/TVDI/趙雲瀚老師/Workplace/Fan_AOI_Austin/Photos/100"]
# base_dir = "C:/OneDrive/! Juxer/TVDI/趙雲瀚老師/Workplace/Project/Fan_AOI_Austin/Photos"
directories = ["Photos/60", "Photos/100"]
classify_images_in_directories(directories)



Classifying images in directory: Photos/60
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 122ms/step
NG (Defective) with 41.48% confidence for image: image-30-B20008.png
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
GO (Correctly Aligned) with 88.81% confidence for image: image-30-B20009.png
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
GO (Correctly Aligned) with 88.69% confidence for image: image-30-B20023.png
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
GO (Correctly Aligned) with 84.94% confidence for image: image-30-B20024.png
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
GO (Correctly Aligned) with 89.31% confidence for image: image-30-B20025.png
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
GO (Correctly Aligned) with 84.36% confidence for image: image-30-B20026.png
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
NG (Defective) with 30.60% confidence for image: image-30-B20027.png
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
NG (Defective) with 32.11% confidence for image: image-30-B20028.png
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
NG (Defective) with 3

### The code needs to import classify function

In [ ]:
import cv2
import numpy as np
import tensorflow as tf
import gradio as gr
from PIL import Image

# Load the pre-trained model
model = tf.keras.models.load_model("fan_model.h5")

# Recompile the model to suppress the warning
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Function to classify a single image
def classify_image(image):
    # Convert the image to a format suitable for OpenCV
    img = cv2.cvtColor(np.array(image), cv2.COLOR_RGB2BGR)
    img = cv2.resize(img, (80, 80))
    img = img.reshape(1, 80, 80, 3) / 255.0  # Normalize
    
    # Make prediction
    prediction = model.predict(img)
    
    # Classification logic
    if prediction is not None and len(prediction) > 0:
        # Set threshold for GO/NG classification
        # Adjusted based on expected results for the provided images
        class_idx = 1 if prediction[0] >= 0.5 else 0  # Threshold set to 0.6
        # confidence = prediction[0][0] * 100  # Confidence score in percentage
        
        if class_idx == 1:
            result_text = f"<div style='color: red; font-size: 24px; font-weight: bold;'>NG</div>"
            return result_text
        else:
            result_text = f"<div style='color: green; font-size: 24px; font-weight: bold;'>GO</div>"
            return result_text
        
    else:
        return "<div style='color: black; font-size: 20px;'>Unable to classify image</div>"

# Create Gradio interface
iface = gr.Interface(
    fn=classify_image,
    inputs=gr.Image(type="pil", label="Upload Image"),
    outputs=gr.HTML(label="Classification Result"),
    title="Cooling Fan GO/NG Inspection",
    description="Upload an image of a cooling fan to check if it is correctly aligned (GO) or defective (NG).",
)

# Launch the interface
iface.launch()


c:\Users\teyin\miniconda3\envs\ForML\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


* Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step


In [ ]:
import gradio as gr
import numpy as np
from PIL import Image

# Function for GO/NO GO classification
def fan_go_ng(image):
    try:
        # Preprocess the input image
        image = np.array(image.resize((80, 80))).astype("float32") / 255.0
        image = image.reshape(1, 80, 80, 3)
        
        # Ensure the model is loaded
        if 'model' not in globals():
            print("Model is not loaded.")
            return '<span style="font-size: 2em; font-weight: bold; color: red;">Model Error</span>'
        
        # Get predictions from the model
        prediction = model.predict(image)
        print("Raw Prediction:", prediction)  # Print raw prediction
        
        # Check if the prediction is a list or array and its shape
        if isinstance(prediction, (list, np.ndarray)):
            prediction = prediction.tolist()
            print("Prediction List:", prediction)
        else:
            print("Unexpected prediction type:", type(prediction))
            return '<span style="font-size: 2em; font-weight: bold; color: red;">Type Error</span>'
        
        # Check if prediction has enough elements
        if len(prediction[0]) < 2:
            print("Prediction length is less than 2.")
            return '<span style="font-size: 2em; font-weight: bold; color: red;">Length Error</span>'
        
        class_names = ["60", "100"]
        
        # Determine GO or NG based on prediction score (assuming "100" class means GO)
        result = "GO" if prediction[0][1] > prediction[0][0] else "NG"
        
        # Return result with color formatting
        color = "green" if result == "GO" else "red"
        return f'<span style="font-size: 2em; font-weight: bold; color: {color};">{result}</span>'
    
    except Exception as e:
        print("Exception caught:", e)
        return '<span style="font-size: 2em; font-weight: bold; color: red;">Error</span>'

# Define Gradio inputs and outputs
inp = gr.Image(type="pil")
out = gr.HTML(label="Result")

# Create the Gradio interface
grobj = gr.Interface(
    fn=fan_go_ng, 
    inputs=inp, 
    outputs=out, 
    title="Cooling Fan GO/NG Inspection", 
    description="Upload or drag-and-drop a fan image to check if it's GO or NG"
)

# Launch the Gradio app
grobj.launch(share=True)


* Running on local URL:  http://127.0.0.1:7862
* Running on public URL: https://c81dc14a592d3c7830.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
Raw Prediction: [[0.9906325]]
Prediction List: [[0.9906324744224548]]
Prediction length is less than 2.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
Raw Prediction: [[0.8577215]]
Prediction List: [[0.8577215075492859]]
Prediction length is less than 2.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
